In [1]:
from IPython.display import clear_output
!pip install pgl
clear_output()
!pip install /home/aistudio/work/rdkit_pypi-2021.3.2-cp37-cp37m-manylinux1_x86_64.whl
clear_output()
print("安装完成")

安装完成


In [2]:
import os
import numpy as np
import sys
sys.path.append("./Transformer/PaddleHelix-dev/")
os.chdir("./Transformer/PaddleHelix-dev/")


In [3]:
import paddle
import paddle.nn as nn
import paddle.distributed as dist
import pgl

from pahelix.model_zoo.pretrain_gnns_model import PretrainGNNModel, AttrmaskModel
from pahelix.datasets.zinc_dataset import load_zinc_dataset
from pahelix.utils.splitters import RandomSplitter
from pahelix.featurizers.pretrain_gnn_featurizer import AttrmaskTransformFn, AttrmaskCollateFn
from pahelix.utils import load_json_config

[INFO] 2022-03-15 09:54:39,672 [mp_reader.py:   20]:	We set multiprocessing start method as 'fork' by default.
[INFO] 2022-03-15 09:54:39,675 [mp_reader.py:   25]:	ujson not install, fail back to use json instead


In [4]:
compound_encoder_config = load_json_config("model_configs/pregnn_paper.json")
model_config = load_json_config("model_configs/pre_Attrmask.json")

compound_encoder = PretrainGNNModel(compound_encoder_config)
model = AttrmaskModel(model_config, compound_encoder)
opt = paddle.optimizer.Adam(0.001, parameters=model.parameters())

[PretrainGNNModel] embed_dim:300
[PretrainGNNModel] dropout_rate:0.5
[PretrainGNNModel] norm_type:batch_norm
[PretrainGNNModel] graph_norm:False
[PretrainGNNModel] residual:False
[PretrainGNNModel] layer_num:5
[PretrainGNNModel] gnn_type:gin
[PretrainGNNModel] JK:last
[PretrainGNNModel] readout:mean
[PretrainGNNModel] atom_names:['atomic_num', 'chiral_tag']
[PretrainGNNModel] bond_names:['bond_dir', 'bond_type']


In [5]:
### Load the first 1000 of the toy dataset for speed up
dataset = load_zinc_dataset("./chem_dataset_small/zinc_standard_agent/")
dataset = dataset[:1000]
print("dataset num: %s" % (len(dataset)))

transform_fn = AttrmaskTransformFn()
dataset.transform(transform_fn, num_workers=2)

dataset num: 1000


Exception in thread Thread-12:
Traceback (most recent call last):
  File "D:\Anaconda3\envs\paddle\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "D:\Anaconda3\envs\paddle\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "D:\Anaconda3\envs\paddle\lib\site-packages\paddle\reader\decorator.py", line 346, in read_worker
    for d in r:
  File "D:\Anaconda3\envs\paddle\lib\site-packages\pgl\utils\mp_reader.py", line 152, in pipe_reader
    p.start()
  File "D:\Anaconda3\envs\paddle\lib\multiprocessing\process.py", line 121, in start
    self._popen = self._Popen(self)
  File "D:\Anaconda3\envs\paddle\lib\multiprocessing\context.py", line 224, in _Popen
    return _default_context.get_context().Process._Popen(process_obj)
  File "D:\Anaconda3\envs\paddle\lib\multiprocessing\context.py", line 327, in _Popen
    return Popen(process_obj)
  File "D:\Anaconda3\envs\paddle\lib\multiprocessing\popen_spawn_win32.py", line 93, in __ini

In [ ]:
def train(model, dataset, collate_fn, opt):
    data_gen = dataset.get_data_loader(
            batch_size=128, 
            num_workers=4, 
            shuffle=True,
            collate_fn=collate_fn)
    list_loss = []
    model.train()
    for graphs, masked_node_indice, masked_node_label in data_gen:
        graphs = graphs.tensor()
        masked_node_indice = paddle.to_tensor(masked_node_indice, 'int64')
        masked_node_label = paddle.to_tensor(masked_node_label, 'int64')
        loss = model(graphs, masked_node_indice, masked_node_label)
        loss.backward()
        opt.step()
        opt.clear_grad()
        list_loss.append(loss.numpy())
    return np.mean(list_loss)

collate_fn = AttrmaskCollateFn(
        atom_names=compound_encoder_config['atom_names'], 
        bond_names=compound_encoder_config['bond_names'],
        mask_ratio=0.15)

for epoch_id in range(2):
    train_loss = train(model, dataset, collate_fn, opt)
    print("epoch:%d train/loss:%s" % (epoch_id, train_loss))
paddle.save(compound_encoder.state_dict(), 
        './model/pretrain_attrmask/compound_encoder.pdparams')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


epoch:0 train/loss:2.8609824
epoch:1 train/loss:0.96538025


In [ ]:
from pahelix.utils.splitters import \
    RandomSplitter, IndexSplitter, ScaffoldSplitter
from pahelix.datasets import *

from src.model import DownstreamModel
from src.featurizer import DownstreamTransformFn, DownstreamCollateFn
from src.utils import calc_rocauc_score, exempt_parameters

In [ ]:
task_names = get_default_tox21_task_names()
# task_names = get_default_sider_task_names()
print(task_names)

['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']


In [ ]:
compound_encoder_config = load_json_config("model_configs/pregnn_paper.json")
model_config = load_json_config("model_configs/down_linear.json")
model_config['num_tasks'] = len(task_names)

compound_encoder = PretrainGNNModel(compound_encoder_config)
model = DownstreamModel(model_config, compound_encoder)
criterion = nn.BCELoss(reduction='none')
opt = paddle.optimizer.Adam(0.001, parameters=model.parameters())

[PretrainGNNModel] embed_dim:300
[PretrainGNNModel] dropout_rate:0.5
[PretrainGNNModel] norm_type:batch_norm
[PretrainGNNModel] graph_norm:False
[PretrainGNNModel] residual:False
[PretrainGNNModel] layer_num:5
[PretrainGNNModel] gnn_type:gin
[PretrainGNNModel] JK:last
[PretrainGNNModel] readout:mean
[PretrainGNNModel] atom_names:['atomic_num', 'chiral_tag']
[PretrainGNNModel] bond_names:['bond_dir', 'bond_type']


In [ ]:
compound_encoder.set_state_dict(paddle.load('./model/pretrain_attrmask/compound_encoder.pdparams'))

In [ ]:
dataset.transform(DownstreamTransformFn(), num_workers=4)

# splitter = RandomSplitter()
splitter = ScaffoldSplitter()
train_dataset, valid_dataset, test_dataset = splitter.split(
        dataset, frac_train=0.8, frac_valid=0.1, frac_test=0.1)
print("Train/Valid/Test num: %s/%s/%s" % (
        len(train_dataset), len(valid_dataset), len(test_dataset)))

RDKit WARNING: [21:21:11] WARNING: not removing hydrogen atom without neighbors
[21:21:11] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [21:21:25] WARNING: not removing hydrogen atom without neighbors
[21:21:25] WARNING: not removing hydrogen atom without neighbors


Train/Valid/Test num: 6264/783/784


In [ ]:
def train(model, train_dataset, collate_fn, criterion, opt):
    data_gen = train_dataset.get_data_loader(
            batch_size=128, 
            num_workers=4, 
            shuffle=True,
            collate_fn=collate_fn)
    list_loss = []
    model.train()
    for graphs, valids, labels in data_gen:
        graphs = graphs.tensor()
        labels = paddle.to_tensor(labels, 'float32')
        valids = paddle.to_tensor(valids, 'float32')
        preds = model(graphs)
        loss = criterion(preds, labels)
        loss = paddle.sum(loss * valids) / paddle.sum(valids)
        loss.backward()
        opt.step()
        opt.clear_grad()
        list_loss.append(loss.numpy())
    return np.mean(list_loss)

def evaluate(model, test_dataset, collate_fn):
    data_gen = test_dataset.get_data_loader(
            batch_size=128, 
            num_workers=4, 
            shuffle=False,
            collate_fn=collate_fn)
    total_pred = []
    total_label = []
    total_valid = []
    model.eval()
    for graphs, valids, labels in data_gen:
        graphs = graphs.tensor()
        labels = paddle.to_tensor(labels, 'float32')
        valids = paddle.to_tensor(valids, 'float32')
        preds = model(graphs)
        total_pred.append(preds.numpy())
        total_valid.append(valids.numpy())
        total_label.append(labels.numpy())
    total_pred = np.concatenate(total_pred, 0)
    total_label = np.concatenate(total_label, 0)
    total_valid = np.concatenate(total_valid, 0)
    return calc_rocauc_score(total_label, total_pred, total_valid)

collate_fn = DownstreamCollateFn(
        atom_names=compound_encoder_config['atom_names'], 
        bond_names=compound_encoder_config['bond_names'])
for epoch_id in range(4):
    train_loss = train(model, train_dataset, collate_fn, criterion, opt)
    val_auc = evaluate(model, valid_dataset, collate_fn)
    test_auc = evaluate(model, test_dataset, collate_fn)
    print("epoch:%s train/loss:%s" % (epoch_id, train_loss))
    print("epoch:%s val/auc:%s" % (epoch_id, val_auc))
    print("epoch:%s test/auc:%s" % (epoch_id, test_auc))
paddle.save(model.state_dict(), './model/tox21/model.pdparams')

Valid ratio: 0.7603235
Task evaluated: 12/12


In [ ]:
compound_encoder_config = load_json_config("model_configs/pregnn_paper.json")
model_config = load_json_config("model_configs/down_linear.json")
model_config['num_tasks'] = len(task_names)

compound_encoder = PretrainGNNModel(compound_encoder_config)
model = DownstreamModel(model_config, compound_encoder)

In [ ]:
model.set_state_dict(paddle.load('./model/model.pdparams'))

In [ ]:
SMILES=""
transform_fn = DownstreamTransformFn(is_inference=True)
collate_fn = DownstreamCollateFn(
        atom_names=compound_encoder_config['atom_names'], 
        bond_names=compound_encoder_config['bond_names'],
        is_inference=True)
graph = collate_fn([transform_fn({'smiles': SMILES})])
preds = model(graph.tensor()).numpy()[0]
print('SMILES:%s' % SMILES)
print('Predictions:')
for name, prob in zip(task_names, preds):
    print("  %s:\t%s" % (name, prob))